In [10]:
import cv2
import os 
import numpy as np 
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from skimage import feature
import itertools

In [7]:
DATASET_PATH = './Dataset_Augmented/'
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 32

In [3]:
print('Getting file paths and labels')

image_paths = []
labels = []

positive_path = os.path.join(DATASET_PATH, 'Positive')
negative_path = os.path.join(DATASET_PATH, 'Negative')

for filename in os.listdir(positive_path):
    image_paths.append(os.path.join(positive_path, filename))
    labels.append(1)
    
for filename in os.listdir(negative_path):
    image_paths.append(os.path.join(negative_path, filename))
    labels.append(0)
    
image_paths = np.array(image_paths)
labels = np.array(labels)

X_train_paths, X_test_paths, y_train, y_test = train_test_split(
    image_paths, labels, test_size=0.25, random_state=42, stratify=labels
)

print(f'Training test size: {len(X_train_paths)}')
print(f'Testing test size: {len(X_test_paths)}')

Getting file paths and labels
Training test size: 1987
Testing test size: 663


In [4]:
NUM_TRAINING_STEPS = len(X_train_paths) // BATCH_SIZE

# Feature Engineering

## Cell Size -> the param that matters the most

In [ ]:
def feature_generator_hog_dynamic(image_paths, labels, batch_size, cell_size):
    num_samples = len(image_paths)
    
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        shuffled_paths = image_paths[indices]
        shuffled_labels = labels[indices]
        
        for i in range (0, num_samples, batch_size):
            batch_paths = shuffled_paths[i:i + batch_size]
            batch_labels = shuffled_labels[i:i + batch_size]
            
            batch_features = []
            
            for img_path in tqdm(batch_paths, desc='Batch Progress'):
                image = cv2.imread(img_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                hog_features = feature.hog(
                    gray_img, orientations=9,
                    pixels_per_cell=cell_size,
                    cells_per_block=(2, 2),
                    transform_sqrt=True,
                    block_norm='L1'
                )
                
                batch_features.append(hog_features)
            
            yield np.array(batch_features), np.array(batch_labels)

In [ ]:
def extract_test_features_hog(paths, cell_size):
    print(f'Extracting features for HOG cell_size: {cell_size}')
    test_features = []
    for img_path in tqdm(paths, desc=f'Testing HOG {cell_size}'):
        image = cv2.imread(img_path)
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        hog_features = feature.hog(
            gray_image, orientations=9,
            pixels_per_cell=cell_size,
            cells_per_block=(2, 2),
            transform_sqrt=True,
            block_norm='L1'
        )
        
        test_features.append(hog_features)
    
    return np.array(test_features)

In [17]:
cell_sizes_to_test = [(8, 8), (16, 16), (32, 32)]
results = {}

print("--- STARTING HOG OPTIMIZATION ---")

for cell_size in cell_sizes_to_test:
    pipeline_name = f"HOG_Cell_{cell_size[0]}x{cell_size[1]}"
    print(f'\n--- Training Model for: {pipeline_name} ---')
    
    model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
    
    train_generator = feature_generator_hog_dynamic(X_train_paths, y_train, BATCH_SIZE, cell_size)
    
    for _ in tqdm(range(int(NUM_TRAINING_STEPS)), desc=f'Training'):
        X_batch, y_batch = next(train_generator)
        model.partial_fit(X_batch, y_batch, classes=np.array([0, 1]))
    
    X_test_features = extract_test_features_hog(X_test_paths, cell_size)
    y_pred = model.predict(X_test_features)
    
    report = classification_report(y_test, y_pred, target_names=['No Crack (0)', 'Crack (1)'], output_dict=True)
    results[pipeline_name] = report

print('\n\n--- HOG OPTIMIZATION RESULTS ---')
best_pipeline = max(results, key=lambda p: results[p]['macro avg']['f1-score'])

for pipeline_name, report in results.items():
    print("==========================================")
    print(f"               {pipeline_name} {'WINNER' if pipeline_name == best_pipeline else ''}")
    print("==========================================")
    print(f"   Accuracy                           {report['accuracy']:.2f}")
    print(f"   Crack F1-Score                     {report['Crack (1)']['f1-score']:.2f}")
    print(f"   Macro Avg F1                       {report['macro avg']['f1-score']:.2f}")

--- STARTING HOG OPTIMIZATION ---

--- Training Model for: HOG_Cell_8x8 ---


Training: 100%|██████████| 62/62 [02:02<00:00,  1.97s/it]


Extracting features for HOG cell_size: (8, 8)


Testing HOG (8, 8): 100%|██████████| 663/663 [00:42<00:00, 15.43it/s]



--- Training Model for: HOG_Cell_16x16 ---


...

Training: 100%|██████████| 62/62 [01:55<00:00,  1.86s/it]


Extracting features for HOG cell_size: (32, 32)


Testing HOG (32, 32): 100%|██████████| 663/663 [00:39<00:00, 16.69it/s]



--- HOG OPTIMIZATION RESULTS ---
               HOG_Cell_8x8 
   Accuracy                           0.68
   Crack F1-Score                     0.68
   Macro Avg F1                       0.68
               HOG_Cell_16x16 WINNER
   Accuracy                           0.72
   Crack F1-Score                     0.72
   Macro Avg F1                       0.72
               HOG_Cell_32x32 
   Accuracy                           0.50
   Crack F1-Score                     0.67
   Macro Avg F1                       0.33



c:\Users\ardel\anaconda3\envs\ml_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ardel\anaconda3\envs\ml_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ardel\anaconda3\envs\ml_env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.s

## All other params

In [12]:
def feature_generator_hog_tuned(image_paths, labels, batch_size, params):
    num_samples = len(image_paths)
    while True:
        indices = np.arange(num_samples); np.random.shuffle(indices)
        shuffled_paths, shuffled_labels = image_paths[indices], labels[indices]
        for i in range(0, num_samples, batch_size):
            batch_paths, batch_labels = shuffled_paths[i:i+batch_size], shuffled_labels[i:i+batch_size]
            batch_features = []
            for img_path in batch_paths:
                image = cv2.imread(img_path); image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                hog_features = feature.hog(gray_image, 
                                           orientations=params['orientations'], 
                                           pixels_per_cell=params['pixels_per_cell'], 
                                           cells_per_block=params['cells_per_block'], 
                                           transform_sqrt=True, 
                                           block_norm=params['block_norm'])
                batch_features.append(hog_features)
            yield np.array(batch_features), np.array(batch_labels)

In [13]:
def extract_test_features_tuned(paths, params):
    test_features = []
    for img_path in tqdm(paths, desc=f"Testing Config", leave=False):
        image = cv2.imread(img_path); image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        hog_features = feature.hog(gray_image, 
                                   orientations=params['orientations'], 
                                   pixels_per_cell=params['pixels_per_cell'], 
                                   cells_per_block=params['cells_per_block'], 
                                   transform_sqrt=True, 
                                   block_norm=params['block_norm'])
        test_features.append(hog_features)
    return np.array(test_features)

In [18]:
param_grid = {
    'orientations': [9, 12],
    'pixels_per_cell': [(16, 16)], 
    'cells_per_block': [(2, 2), (3, 3)],
    'block_norm': ['L1', 'L2-Hys']
}

keys, values = zip(*param_grid.items())
param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

results = {}

print(f"--- STARTING DEEP TUNING ({len(param_combinations)} configurations) ---")

for i, params in enumerate(param_combinations):
    pipeline_name = f"HOG_Conf_{i+1}"
    print(f'\nRunning {pipeline_name}: {params}')
    
    model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
    train_generator = feature_generator_hog_tuned(X_train_paths, y_train, BATCH_SIZE, params)
    
    for _ in tqdm(range(int(NUM_TRAINING_STEPS)), desc=f'Training', leave=False):
        X_batch, y_batch = next(train_generator)
        model.partial_fit(X_batch, y_batch, classes=np.array([0, 1]))
    
    X_test_features = extract_test_features_tuned(X_test_paths, params)
    y_pred = model.predict(X_test_features)
    
    report = classification_report(y_test, y_pred, target_names=['No Crack (0)', 'Crack (1)'], output_dict=True)
    results[pipeline_name] = {'score': report['macro avg']['f1-score'], 'params': params}
    print(f"--> Score: {report['macro avg']['f1-score']:.4f}")

print('\n\n--- HOG TUNING CHAMPION ---')
best_config = max(results, key=lambda p: results[p]['score'])
winner = results[best_config]

print(f"Best Config: {best_config}")
print(f"Macro F1-Score: {winner['score']:.4f}")
print("Parameters:")
for k, v in winner['params'].items():
    print(f"   - {k}: {v}")

--- STARTING DEEP TUNING (8 configurations) ---

Running HOG_Conf_1: {'orientations': 9, 'pixels_per_cell': (16, 16), 'cells_per_block': (2, 2), 'block_norm': 'L1'}


--> Score: 0.7268

Running HOG_Conf_2: {'orientations': 9, 'pixels_per_cell': (16, 16), 'cells_per_block': (2, 2), 'block_norm': 'L2-Hys'}


--> Score: 0.3842

Running HOG_Conf_3: {'orientations': 9, 'pixels_per_cell': (16, 16), 'cells_per_block': (3, 3), 'block_norm': 'L1'}


--> Score: 0.3513

Running HOG_Conf_4: {'orientations': 9, 'pixels_per_cell': (16, 16), 'cells_per_block': (3, 3), 'block_norm': 'L2-Hys'}


--> Score: 0.4478

Running HOG_Conf_5: {'orientations': 12, 'pixels_per_cell': (16, 16), 'cells_per_block': (2, 2), 'block_norm': 'L1'}


--> Score: 0.6879

Running HOG_Conf_6: {'orientations': 12, 'pixels_per_cell': (16, 16), 'cells_per_block': (2, 2), 'block_norm': 'L2-Hys'}


--> Score: 0.3430

Running HOG_Conf_7: {'orientations': 12, 'pixels_per_cell': (16, 16), 'cells_per_block': (3, 3), 'block_norm': 'L1'}


--> Score: 0.6707

Running HOG_Conf_8: {'orientations': 12, 'pixels_per_cell': (16, 16), 'cells_per_block': (3, 3), 'block_norm': 'L2-Hys'}


--> Score: 0.6565


--- HOG TUNING CHAMPION ---
Best Config: HOG_Conf_1
Macro F1-Score: 0.7268
Parameters:
   - orientations: 9
   - pixels_per_cell: (16, 16)
   - cells_per_block: (2, 2)
   - block_norm: L1


{'orientations': 9, 'pixels_per_cell': (16, 16), 'cells_per_block': (2, 2), 'block_norm': 'L1'}

Macro F1-Score: 0.7268

since the score after parameter searching isn't really much, it is decided to just focus on the model selection instead because searching for more will usually just yields to diminishing returns